# Load Data

In [1]:
import gc
import os
import numpy as np
import pandas as pd
import subprocess
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [2]:
def check_fline(fpath):
    """check total number of lines of file for large files
    
    Args:
    fpath: string. file path
    
    Returns:
    None
    
    """
    lines = subprocess.run(['wc', '-l', fpath], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(lines, end='', flush=True)

In [3]:
fs=['./data/train.csv', './data/test.csv', './data/weather_test.csv','./data/weather_train.csv']
[check_fline(s) for s in fs]

20216101 ./data/train.csv
41697601 ./data/test.csv
277244 ./data/weather_test.csv
139774 ./data/weather_train.csv


[None, None, None, None]

In [4]:
# Load sample training data
df_train = pd.read_csv('./data/train.csv')
df_train_weather = pd.read_csv('./data/weather_train.csv')
df_test = pd.read_csv('./data/test.csv')
df_test_weather = pd.read_csv('./data/weather_test.csv')

In [6]:
# Show data shape
print(df_train.shape)
print(df_train_weather.shape)
print(df_test.shape)
print(df_test_weather.shape)

(20216100, 4)
(139773, 9)
(41697600, 4)
(277243, 9)


In [8]:
print(df_train.head())


   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            1      0  2016-01-01 00:00:00            0.0
2            2      0  2016-01-01 00:00:00            0.0
3            3      0  2016-01-01 00:00:00            0.0
4            4      0  2016-01-01 00:00:00            0.0
   site_id            timestamp  air_temperature  cloud_coverage  \
0        0  2016-01-01 00:00:00             25.0             6.0   
1        0  2016-01-01 01:00:00             24.4             NaN   
2        0  2016-01-01 02:00:00             22.8             2.0   
3        0  2016-01-01 03:00:00             21.1             2.0   
4        0  2016-01-01 04:00:00             20.0             2.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0             20.0                NaN              1019.7             0.0   
1             21.1               -1.0              1020.2            70.0   
2           

In [ ]:
df_train_weather.head()